In [ ]:
!mkdir -p ~/agave/funwave-tvd-jenkins-pipeline

%cd ~/agave/funwave-tvd-jenkins-pipeline

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *

!auth-tokens-refresh

# Validating Your Build
* Automation is great, but things can quickly go wrong.
* This is why CI/CD strongly emphasizes good testing practices.
* Testing is central to CI/CD
  * It allows you assess viability of each commit
  * It is the determination of whether or not code can be successfully integrated
  * It allows for code to be automatically deployed to prod, without the direct oversight of a tightly controlled group of developers.
* Production pipelines have multiple testing guards
  * Types of tests: Unit, Functional, Acceptance, Benchmarks, ...

## Adding a Benchmark
* Let's add a simple benchmark to validate performance after each build.
* We'll make a new feature branch for this benchmark

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout -b benchmark"

In [ ]:
writefile("funwave-benchmark-wrapper.txt","""

VERSION=$(cat version.txt | paste -sd "." -)

cd build/benchmark-inputs
docker run funwave-tvd:\${VERSION} <benchmark-command-goes-here>
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-benchmark-wrapper.txt /home/jovyan/FUNWAVE-TVD/build/

In [ ]:
writefile("funwave-benchmark-app.txt","""
{  
   "name":"${AGAVE_USERNAME}-${MACHINE_NAME}-funwave-dbuild",
   "version":"1.0",
   "label":"Benchmarks the funwave docker image",
   "shortDescription":"Funwave docker benchmark",
   "longDescription":"",
   "deploymentSystem":"${AGAVE_STORAGE_SYSTEM_ID}",
   "deploymentPath":"automation/funwave-tvd-docker-automation",
   "templatePath":"build/funwave-benchmark-wrapper.txt",
   "testPath":"test.txt",
   "executionSystem":"${AGAVE_EXECUTION_SYSTEM_ID}",
   "executionType":"CLI",
   "parallelism":"SERIAL",
   "modules":[],
   "inputs":[],
   "parameters":[{
     "id" : "code_version",
     "value" : {
       "visible":true,
       "required":true,
       "type":"string",
       "order":0,
       "enquote":false,
       "default":"latest"
     },
     "details":{
         "label": "Version of the code",
         "description": "If true, output will be packed and compressed",
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     },
     "semantics":{
         "argument": null,
         "showArgument": false,
         "repeatArgument": false
     }
   }],
   "outputs":[]
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F funwave-benchmark-app.txt /home/jovyan/FUNWAVE-TVD/build/

# Automating Our Benchmark

## Add A Jenkinsfile to Run the Benchmark

In [ ]:
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F /home/jovyan/notebooks/build/Jenkinsfile-benchmark /home/jovyan/FUNWAVE-TVD/build/

# DRYing Out Our Jenkinsfiles
Both our build and benchmark Jenkinsfiles are following the same basic workflow _(checkout code, setup Agave CLI, build and submit job to execution environment)_ which has resulted in a bunch of duplicated code that varies only by a keyword. For the sake of maintainability, we should find a way to improve the orthogonality of our pipeline.

## Jenkins Shared Libraries
A [Jenkins Shared Library](https://jenkins.io/doc/book/pipeline/shared-libraries/) is a repository of code that is centrally managed, and made accessible to multiple pipelines. These pipelines can call shared libraries as functions, and pass parameters to specify build options.

## Update the Build Jenkinsfile to Trigger the Benchmark
* We need to add a new stage to the Jenkins pipeline to run the benchmark if the preceeding stages of the build executed successfully.

In [ ]:
writefile("Jenkinsfile", """import groovy.json.JsonOutput

def JobStages = load("JobStages.groovy")


node {
  currentBuild.result = "SUCCESS"

  env.AGAVE_TENANTS_API_BASEURL = "https://sandbox.agaveplatform.org/tenants"
  env.MACHINE_NAME = "sandbox"
  env.AGAVE_APP_NAME = "funwave-tvd-build-\${env.AGAVE_USERNAME}"
  env.AGAVE_CLIENT_NAME = "jenkins-cli-\${env.AGAVE_USERNAME}"

  try {

    JobStages.setUpCli(env)
    JobStages.cloneRepository()
    JobStages.updateApp('build', env)
    JobStages.submitJob('build', env)

    stage("Run benchmarks") {
      // Benchmarking stuff goes here
    }

  } catch (error) {

    currentBuild.result = "FAILURE"
    throw error

  }
}
""")
!files-upload -S ${AGAVE_STORAGE_SYSTEM_ID} -F Jenkinsfile /home/jovyan/FUNWAVE-TVD/build/

# Commit Your Benchmark, Watch It Run
* Let's merge our benchmark back into the `dev` branch and watch it run!

In [ ]:
!ssh sandbox "set -x && cd ~/FUNWAVE-TVD && git add build"
!ssh sandbox "cd ~/FUNWAVE-TVD && git commit -m 'Added benchmark app and pipeline.'"

In [ ]:
!ssh sandbox "cd ~/FUNWAVE-TVD && git checkout dev && git merge --squash benchmark"

# Visualize Our Benchmarks